In [2]:
!pip install datasets

data set duration

In [2]:
from datasets import load_dataset

ds = load_dataset("yskim3271/Throat_and_Acoustic_Pairing_Speech_Dataset")
train_hours = sum(ds["train"]["duration"]) / 3600
dev_hours   = sum(ds["dev"]["duration"]) / 3600
print("train hours:", train_hours)
print("dev hours  :", dev_hours)
print("total hours:", train_hours + dev_hours)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00010.parquet:   0%|          | 0.00/459M [00:00<?, ?B/s]

data/train-00001-of-00010.parquet:   0%|          | 0.00/452M [00:00<?, ?B/s]

data/train-00002-of-00010.parquet:   0%|          | 0.00/576M [00:00<?, ?B/s]

...

data/test-00002-of-00003.parquet:   0%|          | 0.00/388M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

train hours: 10.156891082988844
dev hours  : 2.5018279201454585
total hours: 12.658719003134303


Drive Mount

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installation

In [4]:
!pip -q install -U faster-whisper datasets[audio] soundfile librosa tqdm

Data load & 16k casting

In [7]:
from datasets import load_dataset, Audio

DS_ID = "yskim3271/Throat_and_Acoustic_Pairing_Speech_Dataset"
ACOUSTIC = "audio.acoustic_microphone"

ds = load_dataset(DS_ID)  # train/dev/test
ds["train"] = ds["train"].cast_column(ACOUSTIC, Audio(sampling_rate=16000))
ds["dev"]   = ds["dev"].cast_column(ACOUSTIC, Audio(sampling_rate=16000))

print(ds)
print("train:", len(ds["train"]), "dev:", len(ds["dev"]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['gender', 'speaker_id', 'sentence_id', 'text', 'duration', 'audio.throat_microphone', 'audio.acoustic_microphone'],
        num_rows: 4000
    })
    dev: Dataset({
        features: ['gender', 'speaker_id', 'sentence_id', 'text', 'duration', 'audio.throat_microphone', 'audio.acoustic_microphone'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['gender', 'speaker_id', 'sentence_id', 'text', 'duration', 'audio.throat_microphone', 'audio.acoustic_microphone'],
        num_rows: 1000
    })
})
train: 4000 dev: 1000


Load model

In [9]:
import torch
from faster_whisper import WhisperModel

device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"
model_name = "large-v3"

model = WhisperModel(model_name, device=device, compute_type=compute_type)
print("loaded:", model_name, device, compute_type)


loaded: large-v3 cuda float16


BenchMarking

In [1]:
!pip install torchcodec

In [10]:
import time

def transcribe_one(arr, beam_size=1, vad_filter=False):
    segs, info = model.transcribe(
        arr,
        language="ko",
        task="transcribe",
        beam_size=beam_size,
        vad_filter=vad_filter,
    )
    return "".join(s.text for s in segs).strip()

def benchmark(split="dev", n=100, beam_size=1, vad_filter=False):
    t0 = time.time()
    audio_seconds = 0.0
    for i in range(min(n, len(ds[split]))):
        ex = ds[split][i]
        audio_seconds += float(ex["duration"])
        _ = transcribe_one(ex[ACOUSTIC]["array"], beam_size=beam_size, vad_filter=vad_filter)
    wall = time.time() - t0
    rtf = wall / audio_seconds
    print(f"[benchmark] split={split} n={n} beam={beam_size} vad={vad_filter}")
    print(f"audio_hours={audio_seconds/3600:.3f}, wall_hours={wall/3600:.3f}, RTF={rtf:.3f}")
    total_hours = 12.658719003134303  # 당신이 계산한 total
    print(f"estimated_total_wall_hours ~= {total_hours*rtf:.2f} hours")

benchmark("dev", n=100, beam_size=1, vad_filter=False)


[benchmark] split=dev n=100 beam=1 vad=False
audio_hours=0.266, wall_hours=0.025, RTF=0.094
estimated_total_wall_hours ~= 1.19 hours


Batch Transcription

In [11]:
import os, json
from tqdm import tqdm

OUT_ROOT = "/content/drive/MyDrive/taps_asr/asr_small_acoustic"
os.makedirs(OUT_ROOT, exist_ok=True)

def shard_paths(split, start, end):
    base = os.path.join(OUT_ROOT, split)
    os.makedirs(base, exist_ok=True)
    out_jsonl = os.path.join(base, f"{split}_{start:04d}_{end:04d}.jsonl")
    done_json = os.path.join(base, f"{split}_{start:04d}_{end:04d}.done.json")
    return out_jsonl, done_json

def load_done(path):
    if not os.path.exists(path):
        return set()
    with open(path, "r", encoding="utf-8") as f:
        return set(json.load(f))

def save_done(path, done):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(sorted(list(done)), f)

def transcribe_shard(split, start, end, beam_size=1, vad_filter=False):
    out_jsonl, done_json = shard_paths(split, start, end)
    done = load_done(done_json)

    with open(out_jsonl, "a", encoding="utf-8") as fout:
        for idx in tqdm(range(start, min(end+1, len(ds[split]))), desc=f"{split} {start}-{end}"):
            if idx in done:
                continue

            ex = ds[split][idx]
            text = transcribe_one(ex[ACOUSTIC]["array"], beam_size=beam_size, vad_filter=vad_filter)

            rec = {
                "idx": idx,
                "gender": ex.get("gender"),
                "speaker_id": ex.get("speaker_id"),
                "sentence_id": ex.get("sentence_id"),
                "duration": float(ex.get("duration", 0.0)),
                "raw_asr_text_acoustic": text,
                "beam_size": beam_size,
                "vad_filter": bool(vad_filter),
            }
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
            fout.flush()

            done.add(idx)
            if len(done) % 25 == 0:
                save_done(done_json, done)

    save_done(done_json, done)
    return out_jsonl

def run_split(split, shard_size=200, beam_size=1, vad_filter=False):
    n = len(ds[split])
    for start in range(0, n, shard_size):
        end = min(start + shard_size - 1, n - 1)
        transcribe_shard(split, start, end, beam_size=beam_size, vad_filter=vad_filter)

# 1차 전사(전량): greedy(beam=1)
run_split("train", shard_size=200, beam_size=1, vad_filter=False)
run_split("dev",   shard_size=200, beam_size=1, vad_filter=False)

print("done. outputs at:", OUT_ROOT)


dev 800-999: 100%|██████████| 200/200 [00:00<00:00, 1407484.56it/s]

done. outputs at: /content/drive/MyDrive/taps_asr/asr_small_acoustic


Check Shard

In [12]:
import os, glob

root = "/content/drive/MyDrive/taps_asr/asr_small_acoustic"
for split in ["train","dev"]:
    files = sorted(glob.glob(os.path.join(root, split, f"{split}_*.jsonl")))
    print(split, "jsonl shards:", len(files))
    print(" first:", os.path.basename(files[0]) if files else None)
    print(" last :", os.path.basename(files[-1]) if files else None)


train jsonl shards: 20
 first: train_0000_0199.jsonl
 last : train_3800_3999.jsonl
dev jsonl shards: 5
 first: dev_0000_0199.jsonl
 last : dev_0800_0999.jsonl


In [13]:
import glob, os

def count_lines(path):
    n = 0
    with open(path, "r", encoding="utf-8") as f:
        for _ in f:
            n += 1
    return n

root = "/content/drive/MyDrive/taps_asr/asr_small_acoustic"
for split, expected in [("train", 4000), ("dev", 1000)]:
    files = sorted(glob.glob(os.path.join(root, split, f"{split}_*.jsonl")))
    total = sum(count_lines(p) for p in files)
    print(split, "files:", len(files), "total_lines:", total, "expected:", expected)


train files: 20 total_lines: 4000 expected: 4000
dev files: 5 total_lines: 1000 expected: 1000


Merge shard

In [14]:
import os, glob, shutil

ROOT = "/content/drive/MyDrive/taps_asr/asr_small_acoustic"
MERGED_DIR = os.path.join(ROOT, "_merged")
os.makedirs(MERGED_DIR, exist_ok=True)

def merge_jsonl(split: str):
    src_dir = os.path.join(ROOT, split)
    out_path = os.path.join(MERGED_DIR, f"{split}_1pass_acoustic.jsonl")
    files = sorted(glob.glob(os.path.join(src_dir, f"{split}_*.jsonl")))
    assert files, f"no shards found for {split}"

    with open(out_path, "w", encoding="utf-8") as fout:
        for p in files:
            with open(p, "r", encoding="utf-8") as fin:
                shutil.copyfileobj(fin, fout)

    print("merged:", split, "->", out_path)
    return out_path

train_1pass = merge_jsonl("train")
dev_1pass   = merge_jsonl("dev")


merged: train -> /content/drive/MyDrive/taps_asr/asr_small_acoustic/_merged/train_1pass_acoustic.jsonl
merged: dev -> /content/drive/MyDrive/taps_asr/asr_small_acoustic/_merged/dev_1pass_acoustic.jsonl


In [15]:
!pip -q install -U kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 26.0 MB/s eta 0:00:00


Importing Normalizer

In [16]:
import os

target = "normalizer.py"
hits = []
for root, dirs, files in os.walk("/content/drive/MyDrive"):
    if target in files:
        hits.append(os.path.join(root, target))

hits[:20], len(hits)


(['/content/drive/MyDrive/taps_asr/code/normalizer.py'], 1)

In [17]:
import sys, os

CODE_DIR = "/content/drive/MyDrive/taps_asr/code"
if CODE_DIR not in sys.path:
    sys.path.insert(0, CODE_DIR)

import normalizer
from normalizer import normalize_v063  # 파일 안에 정의된 함수명에 맞춰 사용

print(normalize_v063("도시철도는 정원 120명인 1칸 차량에 180명이 탑승", debug=False))


도시철도는 정원 백 이십 명인 한 칸 차량에 백 팔십 명이 탑승


In [18]:
import normalizer
print(normalizer.__file__)

/content/drive/MyDrive/taps_asr/code/normalizer.py


In [19]:
[name for name in dir(normalizer) if name.startswith("normalize")]

['normalize_numbers_units', 'normalize_v063']

In [20]:
from normalizer import normalize_v063
print(normalize_v063("도시철도는 정원 120명인 1칸 차량에 180명이 탑승했을 경우", debug=False))

도시철도는 정원 백 이십 명인 한 칸 차량에 백 팔십 명이 탑승했을 경우


In [ ]:
import os, glob, shutil, json
from tqdm import tqdm
from normalizer import normalize_v063  

ROOT = "/content/drive/MyDrive/taps_asr/asr_small_acoustic"

# 1) 샤드 병합
MERGED_DIR = os.path.join(ROOT, "_merged")
os.makedirs(MERGED_DIR, exist_ok=True)

def merge_jsonl(split: str):
    src_dir = os.path.join(ROOT, split)
    out_path = os.path.join(MERGED_DIR, f"{split}_1pass_acoustic.jsonl")
    files = sorted(glob.glob(os.path.join(src_dir, f"{split}_*.jsonl")))
    assert files, f"no shards found for {split}"

    with open(out_path, "w", encoding="utf-8") as fout:
        for p in files:
            with open(p, "r", encoding="utf-8") as fin:
                shutil.copyfileobj(fin, fout)

    print("merged:", split, "->", out_path)
    return out_path

train_1pass = merge_jsonl("train")
dev_1pass   = merge_jsonl("dev")

# 2) 정규화 전량 적용
FINAL_DIR = os.path.join(ROOT, "_final")
os.makedirs(FINAL_DIR, exist_ok=True)

def add_normalized(in_jsonl: str, out_jsonl: str, n_lines: int):
    with open(in_jsonl, "r", encoding="utf-8") as fin, \
         open(out_jsonl, "w", encoding="utf-8") as fout:
        for line in tqdm(fin, total=n_lines, desc=f"normalize {os.path.basename(out_jsonl)}"):
            rec = json.loads(line)
            raw = (rec.get("raw_asr_text_acoustic") or "").strip()
            rec["normalized_text"] = normalize_v063(raw, use_kiwi_for_compounds=True, debug=False)
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print("wrote:", out_jsonl)

train_final = os.path.join(FINAL_DIR, "train_acoustic_1pass_normalized.jsonl")
dev_final   = os.path.join(FINAL_DIR, "dev_acoustic_1pass_normalized.jsonl")

add_normalized(train_1pass, train_final, n_lines=4000)
add_normalized(dev_1pass,   dev_final,   n_lines=1000)

print("DONE:", train_final, dev_final)


merged: train -> /content/drive/MyDrive/taps_asr/asr_small_acoustic/_merged/train_1pass_acoustic.jsonl
merged: dev -> /content/drive/MyDrive/taps_asr/asr_small_acoustic/_merged/dev_1pass_acoustic.jsonl


normalize train_acoustic_1pass_normalized.jsonl: 100%|██████████| 4000/4000 [00:03<00:00, 1024.30it/s]


wrote: /content/drive/MyDrive/taps_asr/asr_small_acoustic/_final/train_acoustic_1pass_normalized.jsonl


normalize dev_acoustic_1pass_normalized.jsonl: 100%|██████████| 1000/1000 [00:00<00:00, 2711.93it/s]

wrote: /content/drive/MyDrive/taps_asr/asr_small_acoustic/_final/dev_acoustic_1pass_normalized.jsonl
DONE: /content/drive/MyDrive/taps_asr/asr_small_acoustic/_final/train_acoustic_1pass_normalized.jsonl /content/drive/MyDrive/taps_asr/asr_small_acoustic/_final/dev_acoustic_1pass_normalized.jsonl


In [23]:
import random, json
random.seed(0)
lines = open(dev_final, "r", encoding="utf-8").read().splitlines()
for line in random.sample(lines, 10):
    r = json.loads(line)
    print("\nraw :", r["raw_asr_text_acoustic"])
    print("norm:", r["normalized_text"])



raw : 결국 오시장과 체목사의 면담이 성사돼 중축건물을 기부채나프 사용하는 안에 최종 합의하면서 상황은 일단락됐다.
norm: 결국 오시장과 체목사의 면담이 성사돼 중축건물을 기부채나프 사용하는 안에 최종 합의하면서 상황은 일단락됐다.

raw : 이번 공적과적관계창설은 평생을 얻신한 독립지사에 대한 너무나 당연한 추모의 의식이다.
norm: 이번 공적과적관계창설은 평생을 얻신한 독립지사에 대한 너무나 당연한 추모의 의식이다.

raw : 그러나 A 씨는 불을 지른 후 택시를 타고 현장을 빠져나간 것으로 알려졌다.
norm: 그러나 에이 씨는 불을 지른 후 택시를 타고 현장을 빠져나간 것으로 알려졌다.

raw : 점검을 통해 관리 중인 학대 우려 아동과 보호자를 직접 만나 대면 면담을 하고 주변 이훗의 진술이나 학교 측의 의견을 들어 안전 여부를 몇몇이 확인할 계획이다.
norm: 점검을 통해 관리 중인 학대 우려 아동과 보호자를 직접 만나 대면 면담을 하고 주변 이훗의 진술이나 학교 측의 의견을 들어 안전 여부를 몇몇이 확인할 계획이다.

raw : 노선 버스 기사 5만1300명에게 513억원을 전세 버스 기사 35000명에게 350억원을 지원할 예정이다.
norm: 노선 버스 기사 오만천 삼백 명에게 오백 십 삼억원을 전세 버스 기사 삼만 오천 명에게 삼백 오십억원을 지원할 예정이다.

raw : 실거래 정보와 함께 민간이 조사한 실시 자료 혹가 분양가 등을 고류해 공시 가격을 매긴다는 게 부동상원의 설명이다.
norm: 실거래 정보와 함께 민간이 조사한 실시 자료 혹가 분양가 등을 고류해 공시 가격을 매긴다는 게 부동상원의 설명이다.

raw : 법무부가 세계 난빈의 날 기념 간담회를 열고 균형 잡힌 난민 전책을 추진하겠다고 밝혔다.
norm: 법무부가 세계 난빈의 날 기념 간담회를 열고 균형 잡힌 난민 전책을 추진하겠다고 밝혔다.

raw : 경찰은 A 씨와 K치 그룹 관계자 B 씨를 우선 입건한데 이어 압수수색 자료 분석 등을 토대로 최 전 지